<a href="https://colab.research.google.com/github/sophielu05/114-1-/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E5%8F%8A%E5%88%86%E6%94%A4_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Gradio + Google Sheets 記帳系統（含台灣時間 & 查看紀錄）
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import datetime
import gradio as gr

# --- 時區設定 ---
try:
    from zoneinfo import ZoneInfo
    TAIPEI = ZoneInfo("Asia/Taipei")
except Exception:
    try:
        import pytz
        TAIPEI = pytz.timezone("Asia/Taipei")
    except Exception:
        TAIPEI = datetime.timezone(datetime.timedelta(hours=8))

# --- Google Auth & open sheet ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1AdYsH7xZH7gV7seziToLVL7aKkrc1iIgYmYBvilxACI/edit?usp=sharing'
gsheets = gc.open_by_url(spreadsheet_url)
worksheet = gsheets.get_worksheet(0)  # 第一個工作表

# 若空表就建立表頭
if not worksheet.get_all_values():
    worksheet.append_row(["日期", "時間", "品項", "金額"])

# --- 寫入交易 ---
def record_transaction(item, amount, date="", time=""):
    now = datetime.datetime.now(TAIPEI)

    # 日期
    if not date.strip():
        date_str = now.strftime("%Y-%m-%d")
    else:
        try:
            date_str = datetime.datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d")
        except:
            try:
                date_str = datetime.datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%m-%d")
            except:
                date_str = date

    # 時間
    if not time.strip():
        time_str = now.strftime("%H:%M")
    else:
        time_str = time

    # 金額驗證
    try:
        amt = float(str(amount).replace(",", "").strip())
    except:
        return "❌ 金額必須是數字（可包含逗號），請重新輸入。"

    new_row = [date_str, time_str, item, amt]
    worksheet.append_row(new_row)
    return f"✅ 已新增交易：{date_str} {time_str} — {item}，金額：{amt}"

# --- 讀取所有紀錄 ---
def show_records():
    data = worksheet.get_all_values()
    if not data or len(data) <= 1:
        return pd.DataFrame(columns=["日期", "時間", "品項", "金額"])
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# 💰 記帳系統（Google Sheets 連結版）")

    with gr.Row():
        date_input = gr.Textbox(label="日期 (YYYY-MM-DD 或 YYYYMMDD，留空=今天)")
        time_input = gr.Textbox(label="時間 (HH:MM，留空=現在台灣時間)")
    item_input = gr.Textbox(label="品項")
    amount_input = gr.Textbox(label="金額 (可含逗號)")

    output = gr.Textbox(label="結果")

    with gr.Row():
        submit_btn = gr.Button("新增交易")
        show_btn = gr.Button("📋 查看所有紀錄")

    records_table = gr.DataFrame(label="目前紀錄", interactive=False)

    submit_btn.click(fn=record_transaction,
                     inputs=[item_input, amount_input, date_input, time_input],
                     outputs=output)

    show_btn.click(fn=show_records, inputs=None, outputs=records_table)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://142e2e7de7271e14c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
